In [1]:
import pandas as pd 

df = pd.read_csv('/Users/parth/coding/python/sportly/stats/tennis_atp/singles/atp_matches_net.csv')
df

,Unnamed: 0.1,Unnamed: 0,tourney_name,surface,draw_size,tourney_level,winner_seed,winner_entry,winner_name,winner_hand,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,0,0,Dublin,Grass,32.0,A,NaN,NaN,Doug Smith,U,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,Dublin,Grass,32.0,A,NaN,NaN,Louis Pretorius,R,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,Dublin,Grass,32.0,A,NaN,NaN,Cecil Pedlow,U,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,Dublin,Grass,32.0,A,NaN,NaN,Tom Okker,R,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,Dublin,Grass,32.0,A,NaN,NaN,Armistead Neely,R,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194991,194991,194991,Davis Cup WG2 PO: URU vs MDA,Clay,4.0,D,NaN,NaN,Joaquin Aguilar Cardozo,U,...,30.0,17.0,7.0,6.0,8.0,14.0,1109.0,8.0,740.0,34.0
194992,194992,194992,Davis Cup WG2 PO: VIE vs RSA,Hard,4.0,D,NaN,NaN,Nam Hoang Ly,R,...,41.0,25.0,6.0,9.0,1.0,4.0,554.0,67.0,748.0,32.0
194993,194993,194993,Davis Cup WG2 PO: VIE vs RSA,Hard,4.0,D,NaN,NaN,Kris Van Wyk,R,...,51.0,25.0,7.0,11.0,5.0,12.0,416.0,109.0,NaN,NaN
194994,194994,194994,Davis Cup WG2 PO: VIE vs RSA,Hard,4.0,D,NaN,NaN,Nam Hoang Ly,R,...,51.0,32.0,17.0,14.0,5.0,9.0,554.0,67.0,416.0,109.0


In [2]:
df.head()

,Unnamed: 0.1,Unnamed: 0,tourney_name,surface,draw_size,tourney_level,winner_seed,winner_entry,winner_name,winner_hand,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,0,0,Dublin,Grass,32.0,A,NaN,NaN,Doug Smith,U,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,Dublin,Grass,32.0,A,NaN,NaN,Louis Pretorius,R,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,Dublin,Grass,32.0,A,NaN,NaN,Cecil Pedlow,U,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,Dublin,Grass,32.0,A,NaN,NaN,Tom Okker,R,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,Dublin,Grass,32.0,A,NaN,NaN,Armistead Neely,R,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

In [4]:
str_vals = ['tourney_name', 'surface','tourney_level','winner_hand','winner_ioc','loser_hand','loser_ioc','round','winner_name','loser_name','winner_entry','loser_entry']
label_encoder_variables = []
player_name_map = defaultdict(int)
for variable_name in str_vals:
    var_name = "nlabel_" + variable_name
    globals()[var_name] = LabelEncoder()
    label_encoder_variables.append(globals()[var_name])

In [5]:
for i in range(len(str_vals)):
    if str_vals[i] in df.columns:
        encoded_number = label_encoder_variables[i].fit_transform(df[str_vals[i]])
        df[str_vals[i] + "_n"] = encoded_number
        if str_vals[i] == 'winner_name' or str_vals[i] == 'loser_name':
            player_name_map.update(dict(zip(encoded_number, df[str_vals[i]]))) # pair encoder + winner_player_name


In [6]:
# drop str value columns plus winner name and loser name
for col in str_vals: 
    if col in df.columns:
        df = df.drop(str_vals,axis='columns')
        temp_cols = ['winner_rank','loser_rank','score','Unnamed: 0.1','Unnamed: 0']
        for temp_cols1 in temp_cols:
            if temp_cols1 in df.columns:
                df = df.drop(['winner_rank','loser_rank','score','Unnamed: 0.1','Unnamed: 0'],axis='columns')
df.head()

,draw_size,winner_seed,winner_ht,winner_age,loser_seed,loser_ht,loser_age,best_of,minutes,w_ace,...,tourney_level_n,winner_hand_n,winner_ioc_n,loser_hand_n,loser_ioc_n,round_n,winner_name_n,loser_name_n,winner_entry_n,loser_entry_n
0,32.0,NaN,NaN,NaN,NaN,NaN,24.0,3,NaN,NaN,...,0,3,5,3,61,6,1034,5565,10,11
1,32.0,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,...,0,2,98,3,61,6,2520,4733,10,11
2,32.0,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,...,0,3,54,3,61,6,649,3587,10,11
3,32.0,NaN,178.0,24.3,NaN,NaN,NaN,3,NaN,NaN,...,0,2,81,3,145,6,4090,7071,10,11
4,32.0,NaN,NaN,21.3,NaN,NaN,NaN,3,NaN,NaN,...,0,2,124,3,61,6,335,2742,10,11


In [7]:
player_name_map

{5565: 'Peter Ledbetter',
 4733: 'Maurice Pollock',
 3587: 'John Mulvey',
 7071: 'Unknown Fearmon',
 2742: 'Harry Sheridan',
 945: 'Brendan Kelly',
 3992: 'Kenneth Reid',
 3387: 'Jim Buckley',
 7175: 'Vivian Gotto',
 5575: 'Peter Mockler',
 5436: 'Paul Fitzsimmons',
 2458: 'Gerry Waldron',
 7284: 'Willie Walsh',
 7070: 'Unknown Doherty',
 4811: 'Michael Hickey',
 1643: 'Derek Pollock',
 596: 'Armistead Neely',
 4276: 'Louis Pretorius',
 1651: 'Des Foley',
 2734: 'Harry Barniville',
 1116: 'Cecil Pedlow',
 1791: 'Doug Smith',
 3580: 'John Mcgrath',
 3516: 'John Bartlett',
 2579: 'Graydon Garner',
 866: 'Bob Howe',
 2819: 'Herb Fitzgibbon',
 6890: 'Tom Edlefsen',
 3974: 'Ken Fletcher',
 4554: 'Mark Cox',
 4225: 'Lew Hoad',
 2256: 'Fred Stolle',
 3194: 'Jan Kodes',
 3704: 'Jose Edison Mandarino',
 6819: 'Thomaz Koch',
 480: 'Andres Gimeno',
 6077: 'Rod Laver',
 2649: 'Gustavo Herrero',
 6043: 'Roberto Graetz',
 479: 'Andres Funes',
 1904: 'Emery Neale',
 235: 'Alejandro Harms',
 1087: 'Ca

In [8]:
# rank diff --> negative = loser rank points > winner rank points --> positive = winner rank points > loser rank points
df['rank_points_diff'] = df['winner_rank_points'] - df['loser_rank_points']
df

,draw_size,winner_seed,winner_ht,winner_age,loser_seed,loser_ht,loser_age,best_of,minutes,w_ace,...,winner_hand_n,winner_ioc_n,loser_hand_n,loser_ioc_n,round_n,winner_name_n,loser_name_n,winner_entry_n,loser_entry_n,rank_points_diff
0,32.0,NaN,NaN,NaN,NaN,NaN,24.0,3,NaN,NaN,...,3,5,3,61,6,1034,5565,10,11,NaN
1,32.0,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,...,2,98,3,61,6,2520,4733,10,11,NaN
2,32.0,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,...,3,54,3,61,6,649,3587,10,11,NaN
3,32.0,NaN,178.0,24.3,NaN,NaN,NaN,3,NaN,NaN,...,2,81,3,145,6,4090,7071,10,11,NaN
4,32.0,NaN,NaN,21.3,NaN,NaN,NaN,3,NaN,NaN,...,2,124,3,61,6,335,2742,10,11,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194991,4.0,NaN,NaN,18.8,NaN,188.0,21.8,3,77.0,0.0,...,3,123,2,82,8,2023,2963,10,11,-26.0
194992,4.0,NaN,175.0,26.9,NaN,NaN,23.2,3,131.0,1.0,...,2,127,2,109,8,3021,5634,10,11,35.0
194993,4.0,NaN,185.0,27.3,NaN,183.0,26.4,3,131.0,4.0,...,2,98,2,141,8,2410,4242,10,11,NaN
194994,4.0,NaN,175.0,26.9,NaN,185.0,27.3,3,159.0,0.0,...,2,127,2,109,8,3021,4085,10,11,-42.0


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
import numpy as np
import warnings

warnings.filterwarnings('ignore')

In [10]:
df

,draw_size,winner_seed,winner_ht,winner_age,loser_seed,loser_ht,loser_age,best_of,minutes,w_ace,...,winner_hand_n,winner_ioc_n,loser_hand_n,loser_ioc_n,round_n,winner_name_n,loser_name_n,winner_entry_n,loser_entry_n,rank_points_diff
0,32.0,NaN,NaN,NaN,NaN,NaN,24.0,3,NaN,NaN,...,3,5,3,61,6,1034,5565,10,11,NaN
1,32.0,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,...,2,98,3,61,6,2520,4733,10,11,NaN
2,32.0,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,...,3,54,3,61,6,649,3587,10,11,NaN
3,32.0,NaN,178.0,24.3,NaN,NaN,NaN,3,NaN,NaN,...,2,81,3,145,6,4090,7071,10,11,NaN
4,32.0,NaN,NaN,21.3,NaN,NaN,NaN,3,NaN,NaN,...,2,124,3,61,6,335,2742,10,11,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194991,4.0,NaN,NaN,18.8,NaN,188.0,21.8,3,77.0,0.0,...,3,123,2,82,8,2023,2963,10,11,-26.0
194992,4.0,NaN,175.0,26.9,NaN,NaN,23.2,3,131.0,1.0,...,2,127,2,109,8,3021,5634,10,11,35.0
194993,4.0,NaN,185.0,27.3,NaN,183.0,26.4,3,131.0,4.0,...,2,98,2,141,8,2410,4242,10,11,NaN
194994,4.0,NaN,175.0,26.9,NaN,185.0,27.3,3,159.0,0.0,...,2,127,2,109,8,3021,4085,10,11,-42.0


In [11]:
y = df['winner_name_n']
X = []
for column_name in list(df.columns):
    if column_name != 'winner_name_n' and column_name != 'loser_name_n':
        X.append(column_name)
X = df[X]
X

,draw_size,winner_seed,winner_ht,winner_age,loser_seed,loser_ht,loser_age,best_of,minutes,w_ace,...,surface_n,tourney_level_n,winner_hand_n,winner_ioc_n,loser_hand_n,loser_ioc_n,round_n,winner_entry_n,loser_entry_n,rank_points_diff
0,32.0,NaN,NaN,NaN,NaN,NaN,24.0,3,NaN,NaN,...,2,0,3,5,3,61,6,10,11,NaN
1,32.0,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,...,2,0,2,98,3,61,6,10,11,NaN
2,32.0,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,...,2,0,3,54,3,61,6,10,11,NaN
3,32.0,NaN,178.0,24.3,NaN,NaN,NaN,3,NaN,NaN,...,2,0,2,81,3,145,6,10,11,NaN
4,32.0,NaN,NaN,21.3,NaN,NaN,NaN,3,NaN,NaN,...,2,0,2,124,3,61,6,10,11,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194991,4.0,NaN,NaN,18.8,NaN,188.0,21.8,3,77.0,0.0,...,1,1,3,123,2,82,8,10,11,-26.0
194992,4.0,NaN,175.0,26.9,NaN,NaN,23.2,3,131.0,1.0,...,3,1,2,127,2,109,8,10,11,35.0
194993,4.0,NaN,185.0,27.3,NaN,183.0,26.4,3,131.0,4.0,...,3,1,2,98,2,141,8,10,11,NaN
194994,4.0,NaN,175.0,26.9,NaN,185.0,27.3,3,159.0,0.0,...,3,1,2,127,2,109,8,10,11,-42.0


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xgb_train = xgb.DMatrix(X_train, y_train, enable_categorical=True)
xgb_test = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [20]:
random_forest_classifier = RandomForestClassifier(n_estimators=100,criterion='entropy',max_depth=12,n_jobs=1,random_state=42)

In [21]:
X_train

,draw_size,winner_seed,winner_ht,winner_age,loser_seed,loser_ht,loser_age,best_of,minutes,w_ace,...,surface_n,tourney_level_n,winner_hand_n,winner_ioc_n,loser_hand_n,loser_ioc_n,round_n,winner_entry_n,loser_entry_n,rank_points_diff
187394,32.0,1.0,198.0,26.3,NaN,193.0,28.2,3,111.0,4.0,...,2,0,2,99,2,15,3,10,11,7125.0
178522,4.0,NaN,196.0,27.7,NaN,NaN,27.6,5,143.0,23.0,...,3,1,2,18,2,89,8,10,11,1501.0
155736,32.0,NaN,178.0,26.4,7.0,180.0,34.7,3,137.0,7.0,...,3,0,2,60,2,51,6,10,11,-67.0
91332,32.0,3.0,185.0,20.5,NaN,193.0,23.2,3,59.0,5.0,...,3,0,2,98,2,11,3,10,11,NaN
128844,128.0,NaN,188.0,20.6,NaN,178.0,20.7,5,167.0,11.0,...,1,3,1,36,2,42,4,3,5,140.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,48.0,1.0,190.0,25.6,NaN,180.0,34.0,3,99.0,10.0,...,0,0,2,99,2,137,6,10,5,3606.0
103694,128.0,NaN,183.0,26.5,NaN,180.0,29.9,5,NaN,3.0,...,1,3,1,36,1,64,4,10,11,459.0
131932,32.0,NaN,183.0,22.6,NaN,178.0,26.5,3,94.0,3.0,...,3,0,2,15,2,137,6,10,5,284.0
146867,32.0,NaN,180.0,28.3,NaN,173.0,30.7,3,155.0,12.0,...,3,0,2,98,2,46,6,5,10,-264.0


In [ ]:
random_forest_classifier.fit(X_train,y_train)

In [16]:
y_pred_random_forest = random_forest_classifier.predict(X_test)

In [17]:
y_pred_random_forest

array([1624, 1944,  745, ..., 2180, 1985, 2462])

In [18]:
accuracy = accuracy_score(y_test, y_pred_random_forest)
classification_rep = classification_report(y_test,y_pred_random_forest)

In [19]:
print('accuracy for random forest = ' + str(accuracy))
print('classification report' + str(classification_rep))
df.to_csv('/Users/parth/coding/python/sportly/stats/tennis_atp/singles/atp_matches_net.csv')

accuracy for random forest = 0.5346923076923077
classification report              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         1
          11       0.00      0.00      0.00         1
          12       0.72      0.60      0.65        72
          17       0.00      0.00      0.00         1
          19       0.00      0.00      0.00         1
          21       0.33      0.50      0.40         2
          22       0.00      0.00      0.00         0
          23       0.00      0.00      0.00         1
          26       0.00      0.00      0.00         1
          27       0.33      0.33      0.33         3
          29       0.00      0.00      0.00         1
          31       0.00      0.00      0.00         2
          33       0.00      0.00      0.00         1
          35